In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [ ]:
X_test = pd.

In [ ]:
rf = RandomForestClassifier(max_features=5, n_estimators=100)
rf.fit(X_train, y_train)
rf.predict(X_test)
rf.score(X_test, y_test)